In [ ]:
import numpy as np
import os
import re
import glob

curve_paths = sorted(glob.glob("../Data/Skalerede kurver/*.csv"))
# Læs kurver
aligned_curves = [np.loadtxt(path, delimiter=",") for path in curve_paths]

# Oversæt og hent rørnummer fra filnavn
def translate_filename(filename):
    match = re.search(r"Rør(\d+)Kurve(\d+)", filename)
    if match:
        pipe_num = match.group(1)
        curve_num = match.group(2)
        return f"Pipe {pipe_num} - Suture {curve_num}"
    return filename

labels = [translate_filename(os.path.basename(p).replace("_PC", "")) for p in curve_paths]


# Nu har du aligned_curves i mm 💖


# Længde

In [81]:
def curve_length(curve):
    """
    Beregner den fysiske længde af en kurve i mm, givet koordinater i voxler.
    """
    return np.sum(np.linalg.norm(np.diff(curve, axis=0), axis=1))

# Beregn længder i millimeter
lengths = [curve_length(curve) for curve in aligned_curves]

# Udskriv længder for hver kurve
for name, L in zip(labels, lengths):
    print(f"{name:<25} → Length: {L:.2f} mm")

lengths = np.array(lengths)

# Statistik
print("\n📏 Statistik over kurve-længder (i mm):")
print(f"Min:         {lengths.min():.2f}")
print(f"Maks:        {lengths.max():.2f}")
print(f"Gennemsnit:  {lengths.mean():.2f}")
print(f"Std. afvig:  {lengths.std(ddof=1):.2f}")

Pipe 01 - Suture 4        → Length: 4.72 mm
Pipe 01 - Suture 5        → Length: 4.80 mm
Pipe 02 - Suture 3        → Length: 3.84 mm
Pipe 02 - Suture 4        → Length: 5.03 mm
Pipe 03 - Suture 1        → Length: 4.65 mm
Pipe 03 - Suture 4        → Length: 3.85 mm
Pipe 04 - Suture 1        → Length: 7.10 mm
Pipe 04 - Suture 2        → Length: 4.69 mm
Pipe 05 - Suture 3        → Length: 4.36 mm
Pipe 05 - Suture 4        → Length: 4.15 mm
Pipe 05 - Suture 5        → Length: 4.95 mm
Pipe 06 - Suture 2        → Length: 4.78 mm
Pipe 06 - Suture 4        → Length: 4.78 mm
Pipe 06 - Suture 5        → Length: 4.26 mm
Pipe 07 - Suture 1        → Length: 4.52 mm
Pipe 07 - Suture 2        → Length: 4.75 mm
Pipe 07 - Suture 3        → Length: 5.16 mm
Pipe 07 - Suture 4        → Length: 4.23 mm
Pipe 08 - Suture 1        → Length: 4.37 mm
Pipe 08 - Suture 3        → Length: 5.67 mm
Pipe 08 - Suture 4        → Length: 5.73 mm
Pipe 08 - Suture 5        → Length: 4.96 mm
Pipe 09 - Suture 2        → Leng

In [77]:
plt.figure(figsize=(12, 5))
plt.bar(labels, lengths, color='orchid')
plt.xticks(rotation=90)
plt.axhline(y=264.59, color='black', linestyle='--', linewidth=1.5, label='Mean length')
plt.ylabel("Length [mm]")
plt.title("Length of aligned 3D curves")
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Beregn længder hvis ikke allerede gjort
def curve_length(curve):
    return np.sum(np.linalg.norm(np.diff(curve, axis=0), axis=1))

lengths = [curve_length(curve) for curve in aligned_curves]

# Violinplot med seaborn
plt.figure(figsize=(8, 6))
sns.violinplot(data=lengths, inner='box', orient='h', color='orchid')

plt.title("Fordeling af kraniekurvelængder", fontsize=14)
plt.xlabel("Længde (mm)")
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Dine længder
def curve_length(curve):
    return np.sum(np.linalg.norm(np.diff(curve, axis=0), axis=1))

lengths = [curve_length(curve) for curve in aligned_curves]

plt.figure(figsize=(10, 6))

# Violinplot
sns.violinplot(y=lengths, inner=None, color="skyblue", linewidth=1)

# Stripplot (punkter ovenpå)
sns.stripplot(y=lengths, color="black", size=5, jitter=0.2, alpha=0.7)

# Boxplot (hvis du vil)
sns.boxplot(y=lengths, width=0.15, color="white", showcaps=True, boxprops={'zorder': 2})

plt.title("Raincloud-style plot af kurvelængder")
plt.ylabel("Længde (mm)")
plt.xticks([])  # ingen x-akse da vi kun har én kategori
plt.tight_layout()
plt.show()


# Midter squiggle

In [24]:
def extract_middle_section(curve, fraction=0.16):
    """
    Returnerer den midterste del af kurven.
    fraction=0.3 → midterste 30%
    """
    n = len(curve)
    mid = n // 2
    half_window = int(n * fraction / 2)
    return curve[mid - half_window : mid + half_window]


### Kun midte plot

In [ ]:
import matplotlib.cm as cm
# Plot-indstillinger
n = len(aligned_curves)
rows, cols = 4, 7  # Justér hvis du har flere/færre kurver

fig = plt.figure(figsize=(cols * 3, rows * 3))
n = len(aligned_curves)
cmap = cm.get_cmap('plasma', 40)  # alternativt 'tab10', 'Set1', 'turbo'  # eller 'nipy_spectral', 'viridis' osv.

for i, curve in enumerate(aligned_curves):
    ax = fig.add_subplot(rows, cols, i + 1, projection='3d')
    
    mid = extract_middle_section(curve, fraction=0.25)
    ax.plot(*mid.T, color=cmap(i), linewidth=1.5)
    ax.set_xticks([]); ax.set_yticks([]); ax.set_zticks([])
    ax.set_title(labels[i], fontsize=8)
    ax.set_xlabel("X", fontsize=6)
    ax.set_ylabel("Y", fontsize=6)
    ax.set_zlabel("Z", fontsize=6)
    ax.tick_params(labelsize=6)
    ax.view_init(elev=0, azim=90) 

plt.tight_layout()
plt.suptitle("Middle 'Squiggle' of Each Curve", fontsize=16, y=1.02)
plt.subplots_adjust(top=0.92)
plt.show()

C:\Users\lucca\AppData\Local\Temp\ipykernel_11984\1229060484.py:8: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('plasma', 40)  # alternativt 'tab10', 'Set1', 'turbo'  # eller 'nipy_spectral', 'viridis' osv.


### Midte og lidt ryk plot

In [ ]:
def extract_middle_section(curve, fraction=0.25, shift=0):
    """
    Returnerer midtersektionen af kurven, med valgfri forskydning.
    - fraction: hvor stor del af kurven du vil tage (som andel, fx 0.25)
    - shift: hvor mange punkter forskydes vinduet (positiv = højre, negativ = venstre)
    """
    n = len(curve)
    mid = n // 2 + shift
    half_window = int(n * fraction / 2)
    start = max(0, mid - half_window)
    end = min(n, mid + half_window)
    return curve[start:end]

# Dine manuelle justeringer
# Dine tilpasninger – nu i antal punkter (fx ±20 ud af 1000)
shifts = {
    3:  40,   # ryk lidt til højre (langs kurven)
    10: -40,
    22: -30,
    9: -30,  # ryk lidt til venstre
    20: 40
}


# Udtræk midterstykker med tilpasning (KUN forskel på dem du angiver)
centered_curves = []
for i, curve in enumerate(aligned_curves):
    shift = shifts.get(i, 0)
    section = extract_middle_section(curve, fraction=0.25, shift=shift)
    centered_curves.append(section)

fig = plt.figure(figsize=(cols * 3, rows * 3))
n = len(centered_curves)
cmap = cm.get_cmap('plasma', 40)

for i, curve in enumerate(centered_curves):
    ax = fig.add_subplot(rows, cols, i + 1, projection='3d')
    ax.plot(*curve.T, color=cmap(i), linewidth=1.5)
    ax.set_title(labels[i], fontsize=15)
    ax.set_xlabel("X", fontsize=6)
    ax.set_ylabel("Y", fontsize=6)
    ax.set_zlabel("Z", fontsize=6)
    ax.set_xticks([]); ax.set_yticks([]); ax.set_zticks([])

    ax.tick_params(labelsize=6)
    ax.view_init(elev=0, azim=270)

plt.tight_layout()
plt.suptitle("Middle 'Squiggle' of Each Curve (X-Z view)", fontsize=16, y=1.02)
plt.subplots_adjust(top=0.92)
plt.show()


C:\Users\lucca\AppData\Local\Temp\ipykernel_27376\2304350726.py:34: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('plasma', 40)


In [33]:
middle_lengths= []
amps = np.zeros((27,3))

for i, curve in enumerate(centered_curves):
    length = np.sum(np.linalg.norm(np.diff(centered_curves[i], axis=0), axis=1))
    middle_lengths.append(length)
    amp = np.ptp(centered_curves[i], axis=0)
    amps[i,:] = amp
    #print(f"{labels[i]} → Mid-length: {length:.2f} mm, Amplitude: {amp}")
print(sum(middle_lengths)/len(middle_lengths))
print(np.mean(amps, 0))

63.005867402727965
[19.76983377  1.77154364 23.47870128]


In [40]:
x_amplitudes = [x for x, y, z in amps]
z_amplitudes = [z for x, y, z in amps]

z_x_ratios = [z/x if x > 0 else np.nan for x, z in zip(x_amplitudes, z_amplitudes)]
z_l_ratios = [z/l for z,l in zip(middle_lengths, z_amplitudes)]
z_l_ratios


[2.194448164997575,
 2.4679747113019075,
 3.105137538216443,
 2.2287392610628443,
 2.2163066840245795,
 3.2817129994869205,
 2.442125093430193,
 3.1205847356349796,
 2.7221760815529765,
 2.2984654249753067,
 2.821472194973339,
 3.1595250565847377,
 2.7619492015598266,
 3.023062568565216,
 2.3998836384524243,
 2.429937364550495,
 2.7349634681477046,
 3.01913719039589,
 3.854123569447708,
 2.845827821131648,
 2.444243948967564,
 2.6021563826814864,
 2.308783100404601,
 3.499455649090721,
 2.829227262549003,
 2.479577732111184,
 3.3453151518423705]

# Frekvens på Z-akse

In [ ]:
from scipy.fft import fft, fftfreq
components = ['X', 'Y', 'Z']
plt.figure(figsize=(16, 10))

for j, axis in enumerate([0, 1, 2]):  # 0 = X, 1 = Y, 2 = Z
    plt.subplot(3, 1, j + 1)
    
    for i, curve in enumerate(aligned_curves):
        signal = curve[:, axis]
        N = len(signal)
        L = curve_length(curve)
        spacing = L / N

        F = fft(signal)
        freqs_mm = fftfreq(N, d=spacing)
        idx = (freqs_mm > 0) & (freqs_mm <= 0.1)  # zoom

        spectrum = 2.0 / N * np.abs(F[idx])
        plt.plot(freqs_mm[idx], spectrum, label=labels[i], alpha=0.4)

    plt.title(f"Frekvensspektrum af {components[j]}-komponent (0–0.1 [1/mm])")
    plt.xlabel("Frekvens [1/mm]")
    plt.ylabel("Amplitude")
    plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
print("Dominerende frekvens pr. komponent (i [1/mm]):\n")

for i, curve in enumerate(aligned_curves):
    dom_freqs = []
    N = len(curve)
    L = curve_length(curve)
    spacing = L / N

    for axis in range(3):
        signal = curve[:, axis]
        F = fft(signal)
        freqs_mm = fftfreq(N, d=spacing)
        idx = freqs_mm > 0
        dominant_freq = freqs_mm[idx][np.argmax(np.abs(F[idx]))]
        dom_freqs.append(dominant_freq)
    
    print(f"{labels[i]:<25} → X: {dom_freqs[0]:.4f}, Y: {dom_freqs[1]:.4f}, Z: {dom_freqs[2]:.4f}")


Dominerende frekvens pr. komponent (i [1/mm]):

Pipe 01 – Curve 4         → X: 0.0040, Y: 0.0040, Z: 0.0160
Pipe 01 – Curve 5         → X: 0.0039, Y: 0.0039, Z: 0.0039
Pipe 02 – Curve 3         → X: 0.0044, Y: 0.0044, Z: 0.0044
Pipe 02 – Curve 4         → X: 0.0034, Y: 0.0034, Z: 0.0135
Pipe 03 – Curve 1         → X: 0.0047, Y: 0.0047, Z: 0.0189
Pipe 03 – Curve 4         → X: 0.0057, Y: 0.0057, Z: 0.0057
Pipe 04 – Curve 2         → X: 0.0026, Y: 0.0026, Z: 0.0104
Pipe 05 – Curve 3         → X: 0.0045, Y: 0.0045, Z: 0.0045
Pipe 05 – Curve 4         → X: 0.0047, Y: 0.0047, Z: 0.0047
Pipe 05 – Curve 5         → X: 0.0039, Y: 0.0039, Z: 0.0158
Pipe 06 – Curve 2         → X: 0.0038, Y: 0.0038, Z: 0.0077
Pipe 06 – Curve 4         → X: 0.0038, Y: 0.0038, Z: 0.0038
Pipe 06 – Curve 5         → X: 0.0043, Y: 0.0043, Z: 0.0043
Pipe 07 – Curve 1         → X: 0.0040, Y: 0.0040, Z: 0.0040
Pipe 07 – Curve 2         → X: 0.0039, Y: 0.0039, Z: 0.0154
Pipe 07 – Curve 3         → X: 0.0035, Y: 0.0035, Z:

In [ ]:
print(f"{'Kurve':<30} {'Frekvens [1/mm]':>18} {'Bølgelængde [mm]':>20} {'Kurvelængde [mm]':>20} {'Antal bølger':>15}")
print("-" * 105)

for i, curve in enumerate(aligned_curves):
    z = curve[:, 2]
    N = len(z)
    L = curve_length(curve)  # total længde i mm
    spacing = L / N

    Z = fft(z)
    freqs = fftfreq(N, d=spacing)
    idx = freqs > 0
    dominant_freq = freqs[idx][np.argmax(np.abs(Z[idx]))]
    wavelength = 1 / dominant_freq if dominant_freq > 0 else float('inf')
    num_waves = L / wavelength if wavelength > 0 else 0

    print(f"{labels[i]:<30} {dominant_freq:>18.4f} {wavelength:>20.2f} {L:>20.2f} {num_waves:>15.2f}")


Kurve                             Frekvens [1/mm]     Bølgelængde [mm]     Kurvelængde [mm]    Antal bølger
---------------------------------------------------------------------------------------------------------
Pipe 01 – Curve 4                          0.0160                62.39               249.57            4.00
Pipe 01 – Curve 5                          0.0039               253.51               253.51            1.00
Pipe 02 – Curve 3                          0.0044               226.69               226.69            1.00
Pipe 02 – Curve 4                          0.0135                74.19               296.75            4.00
Pipe 03 – Curve 1                          0.0189                53.00               211.98            4.00
Pipe 03 – Curve 4                          0.0057               175.71               175.71            1.00
Pipe 04 – Curve 2                          0.0104                96.35               385.41            4.00
Pipe 05 – Curve 3             

In [ ]:
from scipy.signal import find_peaks

for i, curve in enumerate(aligned_curves):
    z = curve[:, 2]
    peaks, _ = find_peaks(z)
    valleys, _ = find_peaks(-z)
    num_waves = (len(peaks) + len(valleys)) / 2

    print(f"{labels[i]:<30} → Estimerede bølger i Z: {num_waves:.1f}")


Pipe 01 – Curve 4              → Estimerede bølger i Z: 6.5
Pipe 01 – Curve 5              → Estimerede bølger i Z: 5.5
Pipe 02 – Curve 3              → Estimerede bølger i Z: 4.5
Pipe 02 – Curve 4              → Estimerede bølger i Z: 4.5
Pipe 03 – Curve 1              → Estimerede bølger i Z: 4.5
Pipe 03 – Curve 4              → Estimerede bølger i Z: 3.5
Pipe 04 – Curve 2              → Estimerede bølger i Z: 3.5
Pipe 05 – Curve 3              → Estimerede bølger i Z: 3.5
Pipe 05 – Curve 4              → Estimerede bølger i Z: 3.5
Pipe 05 – Curve 5              → Estimerede bølger i Z: 4.5
Pipe 06 – Curve 2              → Estimerede bølger i Z: 3.5
Pipe 06 – Curve 4              → Estimerede bølger i Z: 3.5
Pipe 06 – Curve 5              → Estimerede bølger i Z: 3.5
Pipe 07 – Curve 1              → Estimerede bølger i Z: 3.5
Pipe 07 – Curve 2              → Estimerede bølger i Z: 4.5
Pipe 07 – Curve 3              → Estimerede bølger i Z: 3.5
Pipe 07 – Curve 4              → Estimer

# Lokal maks og min

In [ ]:
from scipy.signal import find_peaks

fig = plt.figure(figsize=(cols * 3, rows * 3))

for i, curve in enumerate(aligned_curves):
    x, y, z = curve.T
    peaks, _ = find_peaks(z)
    valleys, _ = find_peaks(-z)

    ax = fig.add_subplot(rows, cols, i + 1, projection='3d')
    ax.plot(x, y, z, color='gray', linewidth=1.0)
    ax.scatter(x[peaks], y[peaks], z[peaks], color='red', marker='^', label='Toppe')
    ax.scatter(x[valleys], y[valleys], z[valleys], color='blue', marker='v', label='Dale')

    ax.set_title(labels[i], fontsize=8)
    ax.set_xlabel("X", fontsize=6)
    ax.set_ylabel("Y", fontsize=6)
    ax.set_zlabel("Z", fontsize=6)
    ax.tick_params(labelsize=6)
    ax.view_init(elev=0, azim=90)

# Justering
plt.tight_layout()
plt.suptitle("3D-visning af kurver med Z-toppe og -dale", fontsize=14, y=1.02)
plt.subplots_adjust(top=0.92)
plt.show()


# Gennemsnitskurve

### Lav den

#### Første reparametrisering

In [ ]:
from scipy.interpolate import interp1d

def reparameterize_by_arclength(curve, num_points=1000):
    # 1. Beregn kumulativ længde
    deltas = np.diff(curve, axis=0)
    segment_lengths = np.linalg.norm(deltas, axis=1)
    cumulative_length = np.insert(np.cumsum(segment_lengths), 0, 0)
    total_length = cumulative_length[-1]
    s = cumulative_length / total_length  # normaliseret arc length i [0,1]

    # 2. Interpolér X, Y, Z som funktion af s
    x_interp = interp1d(s, curve[:, 0], kind='linear')
    y_interp = interp1d(s, curve[:, 1], kind='linear')
    z_interp = interp1d(s, curve[:, 2], kind='linear')

    s_uniform = np.linspace(0, 1, num_points)
    reparam_curve = np.stack([
        x_interp(s_uniform),
        y_interp(s_uniform),
        z_interp(s_uniform)
    ], axis=1)

    return reparam_curve
# Reparameteriser alle kurver efter arc length
reparam_curves = [reparameterize_by_arclength(c, num_points=1000) for c in aligned_curves]

# Brug X fra den første kurve som reference
ref_x = reparam_curves[0][:, 0]

# Udtræk Y og Z fra alle kurver og tag gennemsnit
X_all = np.stack([c[:, 1] for c in reparam_curves])
Y_all = np.stack([c[:, 1] for c in reparam_curves])
Z_all = np.stack([c[:, 2] for c in reparam_curves])

mean_x = np.mean(X_all, axis=0)
mean_y = np.mean(Y_all, axis=0)
mean_z = np.mean(Z_all, axis=0)

# Saml mean-kurven
mean_curve = np.stack([mean_x, mean_y, mean_z], axis=1)


#### Anden reparametrisering

In [ ]:
import numpy as np
from scipy.interpolate import interp1d

def reparameterize_by_arclength(curve, num_points=1000):
    deltas = np.diff(curve, axis=0)
    segment_lengths = np.linalg.norm(deltas, axis=1)
    cumulative_length = np.insert(np.cumsum(segment_lengths), 0, 0)
    total_length = cumulative_length[-1]
    s = cumulative_length / total_length  # normaliseret parameter [0,1]

    x_interp = interp1d(s, curve[:, 0], kind='linear')
    y_interp = interp1d(s, curve[:, 1], kind='linear')
    z_interp = interp1d(s, curve[:, 2], kind='linear')

    s_uniform = np.linspace(0, 1, num_points)
    reparam_curve = np.stack([
        x_interp(s_uniform),
        y_interp(s_uniform),
        z_interp(s_uniform)
    ], axis=1)
    
    return reparam_curve

def compute_mean_curve(curves, num_points=1000, average_x=True):
    """
    Beregn en mean-kurve ud fra en liste af 3D-kurver.
    
    Parametre:
        curves: list of (N,3) numpy arrays
        num_points: antal punkter på outputkurver
        average_x: hvis False, bruges X fra første kurve som reference
    
    Returnerer:
        mean_curve: (num_points, 3) numpy array
    """

    # Reparameterisér alle kurver
    reparam_curves = [reparameterize_by_arclength(c, num_points=num_points) for c in curves]

    # Udtræk koordinater
    X_all = np.stack([c[:, 0] for c in reparam_curves])
    Y_all = np.stack([c[:, 1] for c in reparam_curves])
    Z_all = np.stack([c[:, 2] for c in reparam_curves])

    # Beregn mean
    if average_x:
        mean_x = np.mean(X_all, axis=0)
    else:
        mean_x = reparam_curves[0][:, 0]  # referencekurvens X

    mean_y = np.mean(Y_all, axis=0)
    mean_z = np.mean(Z_all, axis=0)

    # Saml mean-kurven
    mean_curve = np.stack([mean_x, mean_y, mean_z], axis=1)

    return mean_curve

# curves er din liste af alignede kurver, fx: aligned_curves eller rotated_curves
mean_curve = compute_mean_curve(curves=aligned_curves, num_points=1000, average_x=True)


#### Flip dem som er spejlvendt

In [ ]:

# - aligned_curves: dine kurver
# - flip_indices: de kurver der skal "vendes om" (reverseres)
flip_indices = [1, 2, 3, 5, 6, 9, 13, 21, 23, 25, 26]
final_curves = []

for i, curve in enumerate(aligned_curves):
    if i in flip_indices:
        flipped = curve[::-1]  # vend rækkefølgen af punkterne
        final_curves.append(flipped)
        #print(f"🔁 Vendte rækkefølgen på kurve {i}")
    else:
        final_curves.append(curve)



In [ ]:
# curves: Liste af (1000 x 3) arrays — allerede resamplet!
# Brug X fra den første kurve som reference
ref_x = final_curves[0][:, 0]

# Udtræk Y og Z fra alle aligned_curves
X_all = np.stack([curve[:, 0] for curve in final_curves])
Y_all = np.stack([curve[:, 1] for curve in final_curves])
Z_all = np.stack([curve[:, 2] for curve in final_curves])

# Beregn gennemsnit for hvert punkt langs kurven
mean_x = np.mean(X_all, axis=0)
mean_y = np.mean(Y_all, axis=0)
mean_z = np.mean(Z_all, axis=0)

# Saml den endelige mean-kurve
mean_curve = np.stack([mean_x, mean_y, mean_z], axis=1)


In [ ]:
%matplotlib qt

### Suplots

In [ ]:
import matplotlib.cm as cm
# Plot-indstillinger
n = len(aligned_curves)
rows, cols = 4, 7  # Justér hvis du har flere/færre kurver

fig = plt.figure(figsize=(cols * 3, rows * 3))
n = len(aligned_curves)
cmap = cm.get_cmap('plasma', 40)  # alternativt 'tab10', 'Set1', 'turbo'  # eller 'nipy_spectral', 'viridis' osv.

for i, curve in enumerate(aligned_curves):
    ax = fig.add_subplot(rows, cols, i + 1, projection='3d')
    ax.plot(*curve.T, color = cmap(i), linewidth=1.0, alpha=0.7)
    ax.plot(*mean_curve.T, color='black', linewidth=2.0)
    
    ax.set_title(labels[i], fontsize=8)
    ax.tick_params(labelsize=6)
    ax.set_xlabel("X", fontsize=6)
    ax.set_ylabel("Y", fontsize=6)
    ax.set_zlabel("Z", fontsize=6)
    ax.view_init(elev=0, azim=270)

plt.tight_layout()
plt.suptitle("Kurver reparameteriseret efter arc length – med gennemsnit", fontsize=14, y=1.02)
plt.subplots_adjust(top=0.92)
plt.show()


C:\Users\lucca\AppData\Local\Temp\ipykernel_27376\1929327773.py:8: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('plasma', 40)  # alternativt 'tab10', 'Set1', 'turbo'  # eller 'nipy_spectral', 'viridis' osv.


### Alle i et

In [ ]:
# Brug dine egne kurver i stedet for dummy data

# Plot alle rigtige kurver + mean
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')

for c in aligned_curves:
    ax.plot(c[:, 0], c[:, 1], c[:, 2], color='gray', alpha=0.3)

ax.plot(mean_curve[:, 0], mean_curve[:, 1], mean_curve[:, 2], color='black', linewidth=2, label='Mean Suture')
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.legend(prop={'size': 15})
plt.tight_layout()
plt.show()



### Stat

In [46]:
import pandas as pd
import numpy as np

# Statistik for mean-kurven
mean_diffs = np.diff(mean_curve, axis=0)
mean_lengths = np.linalg.norm(mean_diffs, axis=1)
mean_total_length = np.sum(mean_lengths)
mean_var = np.var(mean_curve, axis=0,ddof=1)
mean_std = np.std(mean_curve, axis=0,ddof=1)

# Saml i DataFrame
mean_stats = pd.DataFrame({
    "Coordinate": ["X", "Y", "Z"],
    "Mean": np.mean(mean_curve, axis=0),
    "Variance": mean_var,
    "Std Deviation": mean_std
})
mean_stats.loc[len(mean_stats)] = ["Total Length", mean_total_length, None, None]

# Print pænt
print("\nStatistik for mean-kurven:\n")
print(mean_stats.to_string(index=False))



Statistik for mean-kurven:

  Coordinate       Mean   Variance  Std Deviation
           X  -0.497077 993.012331      31.512098
           Y   0.714500  54.752931       7.399522
           Z  -0.518691  84.609038       9.198317
Total Length 213.763215        NaN            NaN


C:\Users\lucca\AppData\Local\Temp\ipykernel_27376\42327952.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mean_stats.loc[len(mean_stats)] = ["Total Length", mean_total_length, None, None]


In [ ]:
# Brug mean/std for X, Y og Z
X_all = np.stack([c[:, 0] for c in aligned_curves])
mean_x = np.mean(X_all, axis=0)
mean_y = np.mean(Y_all, axis=0)
mean_z = np.mean(Z_all, axis=0)

std_x = np.std(X_all, axis=0, ddof=1)
std_y = np.std(Y_all, axis=0, ddof=1)
std_z = np.std(Z_all, axis=0, ddof=1)

arc = np.linspace(0, 1, mean_x.shape[0])

# Plot X, Y, Z mean ± std
fig, axs = plt.subplots(3, 1, figsize=(10, 9), sharex=True)

# X
axs[0].plot(arc, mean_x, color='black', label='Mean X')
axs[0].fill_between(arc, mean_x - std_x, mean_x + std_x, color='orchid', alpha=0.3, label='±1 Std')
axs[0].set_ylabel("X")
axs[0].set_title("Mean ± Std Deviation")
axs[0].grid(True)
axs[0].legend()

# Y
axs[1].plot(arc, mean_y, color='black', label='Mean Y')
axs[1].fill_between(arc, mean_y - std_y, mean_y + std_y, color='orchid', alpha=0.3, label='±1 Std')
axs[1].set_ylabel("Y")
axs[1].grid(True)
axs[1].legend()

# Z
axs[2].plot(arc, mean_z, color='black', label='Mean Z')
axs[2].fill_between(arc, mean_z - std_z, mean_z + std_z, color='orchid', alpha=0.3, label='±1 Std')
axs[2].set_ylabel("Z")
axs[2].set_xlabel("Arc Length Position")
axs[2].grid(True)
axs[2].legend()

plt.tight_layout()
plt.show()


1000


### Error fra gennemsnitskurve

In [15]:
rms_errors = []

for c in final_curves:
    diff = c - mean_curve  # forskel i alle koordinater
    rms = np.sqrt(np.mean(np.sum(diff**2, axis=1)))  # ||v||^2 → mean → sqrt
    rms_errors.append(rms)
for i, rms in enumerate(rms_errors):
    print(f"{labels[i]:<25} → RMS-fejl til mean: {rms:.4f}")


Pipe 01 – Curve 4         → RMS-fejl til mean: 7.0321
Pipe 01 – Curve 5         → RMS-fejl til mean: 5.7437
Pipe 02 – Curve 3         → RMS-fejl til mean: 5.0970
Pipe 02 – Curve 4         → RMS-fejl til mean: 10.8473
Pipe 03 – Curve 1         → RMS-fejl til mean: 9.1504
Pipe 03 – Curve 4         → RMS-fejl til mean: 7.5075
Pipe 04 – Curve 2         → RMS-fejl til mean: 17.7979
Pipe 05 – Curve 3         → RMS-fejl til mean: 4.6000
Pipe 05 – Curve 4         → RMS-fejl til mean: 4.3932
Pipe 05 – Curve 5         → RMS-fejl til mean: 5.9452
Pipe 06 – Curve 2         → RMS-fejl til mean: 9.1767
Pipe 06 – Curve 4         → RMS-fejl til mean: 12.0541
Pipe 06 – Curve 5         → RMS-fejl til mean: 4.8188
Pipe 07 – Curve 1         → RMS-fejl til mean: 8.6903
Pipe 07 – Curve 2         → RMS-fejl til mean: 4.0473
Pipe 07 – Curve 3         → RMS-fejl til mean: 4.5522
Pipe 07 – Curve 4         → RMS-fejl til mean: 9.4874
Pipe 08 – Curve 1         → RMS-fejl til mean: 2.6980
Pipe 08 – Curve 3        

### Bedste tre og værste tre

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


# Sort and select best and worst
indexed_curves = list(zip(labels, aligned_curves, rms_errors))
sorted_curves = sorted(indexed_curves, key=lambda x: x[2])
best = sorted_curves[:3]
worst = sorted_curves[-3:]

# Plot in 3D
fig = plt.figure(figsize=(15, 10))
titles = ["Best 1", "Best 2", "Best 3", "Worst 3", "Worst 2", "Worst 1"]

for i, (label, curve, _) in enumerate(best + worst[::-1]):
    ax = fig.add_subplot(2, 3, i+1, projection='3d')
    ax.plot(curve[:, 0], curve[:, 1], curve[:, 2], label='Original', linestyle='--', color = "orchid")
    ax.plot(mean_curve[:, 0], mean_curve[:, 1], mean_curve[:, 2], color='black', label='Mean')
    ax.set_title(f"{titles[i]}: {label}", fontsize=15)
    ax.set_xticks([]); ax.set_yticks([]); ax.set_zticks([])
    ax.set_xlabel("X", fontsize=7)
    ax.set_ylabel("Y", fontsize=7)
    ax.set_zlabel("Z", fontsize=7)
    #ax.view_init(elev=30, azim=45)
    ax.view_init(elev=0, azim=90)
    ax.grid(True)

plt.tight_layout(pad=0.5)
plt.subplots_adjust(top=0.88)
plt.show()


# Næste